In [1]:
import pandas as pd
import numpy as np
import os
import nltk
pd.set_option('display.max_colwidth', 1000)

In [2]:
def analyze(filename):
    df = pd.read_csv(filename, header=-1)
    df.columns = ['pred', 'label'] # Distribution of templates
    
    for id in df['label'].value_counts().index:
        subset = df[df['label'] == id]
        correct = (subset['pred'] == subset['label']).sum()
        accuracy = correct / len(subset)

        false_positive_subset = df[df['pred'] == id]
        false_positives = (false_positive_subset['pred'] != false_positive_subset['label']).sum()
        false_positive_rate = false_positives / max(len(false_positive_subset), 1)

        print("=" * 50)
        print("Template #{}: count={}, correct={}, false_positives={}, accuracy={}, false_positive_rate={}"
              .format(id, len(subset), correct, false_positives, accuracy, false_positive_rate))
        print("=" * 50)
    return df

In [3]:
df = analyze("analysis/expname=reverting to old ways with lower learning rate,input_dim=444,mem_dim=150,lr=0.001,emblr=0.01,wd=0.0001,epochs=50,current_epoch=8,test_acc=0.786,loss=0.6967378057959431.csv")

Template #5: count=154, correct=122, false_positives=52, accuracy=0.7922077922077922, false_positive_rate=0.2988505747126437
Template #2: count=149, correct=119, false_positives=22, accuracy=0.7986577181208053, false_positive_rate=0.15602836879432624
Template #1: count=112, correct=72, false_positives=20, accuracy=0.6428571428571429, false_positive_rate=0.21739130434782608
Template #16: count=104, correct=93, false_positives=13, accuracy=0.8942307692307693, false_positive_rate=0.12264150943396226
Template #8: count=79, correct=75, false_positives=12, accuracy=0.9493670886075949, false_positive_rate=0.13793103448275862
Template #151: count=73, correct=71, false_positives=2, accuracy=0.9726027397260274, false_positive_rate=0.0273972602739726
Template #3: count=69, correct=50, false_positives=23, accuracy=0.7246376811594203, false_positive_rate=0.3150684931506849
Template #6: count=51, correct=28, false_positives=14, accuracy=0.5490196078431373, false_positive_rate=0.3333333333333333
Temp

In [149]:
(sum(df['pred'] == df['label']) + 45) / len(df)

0.8835341365461847

In [142]:
template = 102
df[ (df['pred'] == template) & (df['label'] != template) ]['label'].value_counts()

Series([], Name: label, dtype: int64)

In [4]:
# Reverse
template = 1
df[ (df['label'] == template) & (df['pred'] != template) ]['pred'].value_counts()

2    12
5    11
6     8
8     6
7     2
3     1
Name: pred, dtype: int64

In [23]:
df = pd.read_csv('data/lc-quad/dataset.csv')

In [12]:
with open('data/lc-quad/train/output.txt') as f:
    x = f.read().split('\n')
    
s = pd.Series(x)
s.value_counts()

5      622
2      582
16     415
1      378
3      307
8      300
151    291
6      214
105    211
15     168
101    119
7      108
11      80
106     79
111     56
103     25
102     14
108     11
         1
dtype: int64